In [2]:
from google.colab import files
uploaded = files.upload()

Saving NSE-RELIANCE.csv to NSE-RELIANCE.csv


In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('NSE-RELIANCE.csv')

In [5]:
import datetime
df['Date']= pd.to_datetime(df['Date']) 
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['weekday'] = df['Date'].dt.weekday
df['weekday_name'] = df['Date'].dt.day_name()
df.head(5)

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs),year,month,day,weekday,weekday_name
0,2019-01-04,1097.40,1104.45,1081.1,1099.60,1098.65,8465141.0,92529.47,2019,1,4,4,Friday
1,2019-01-03,1107.50,1114.60,1090.1,1090.30,1092.75,7446457.0,82012.92,2019,1,3,3,Thursday
2,2019-01-02,1114.50,1127.00,1101.0,1104.35,1106.40,7144970.0,79553.42,2019,1,2,2,Wednesday
3,2019-01-01,1125.25,1127.30,1110.1,1120.50,1121.00,4455850.0,49861.29,2019,1,1,1,Tuesday
4,2018-12-31,1130.95,1133.85,1116.2,1125.25,1121.25,7222817.0,81060.63,2018,12,31,0,Monday


In [6]:
new_df1=pd.DataFrame(df,columns=['day','month','year','weekday','weekday_name','Close'])
new_df1.tail()

,day,month,year,weekday,weekday_name,Close
5179,26,3,1998,3,Thursday,179.45
5180,25,3,1998,2,Wednesday,183.85
5181,24,3,1998,1,Tuesday,178.70
5182,23,3,1998,0,Monday,180.20
5183,20,3,1998,4,Friday,NaN


In [7]:
new_df1.fillna(0)

,day,month,year,weekday,weekday_name,Close
0,4,1,2019,4,Friday,1098.65
1,3,1,2019,3,Thursday,1092.75
2,2,1,2019,2,Wednesday,1106.40
3,1,1,2019,1,Tuesday,1121.00
4,31,12,2018,0,Monday,1121.25
...,...,...,...,...,...,...
5179,26,3,1998,3,Thursday,179.45
5180,25,3,1998,2,Wednesday,183.85
5181,24,3,1998,1,Tuesday,178.70
5182,23,3,1998,0,Monday,180.20


In [8]:
from dateutil.relativedelta import relativedelta
datee= pd.to_datetime(df['Date'][0]) 
yy= datee.year
m= datee.month
d= datee.day
last_two_months = datetime.date(yy,m,d)+ relativedelta(months=-2)

In [9]:
d1=df['Date'][0]
d2=last_two_months
length=0
for i in df['Date']:
    if i>=d2 and i<=d1:
        length+=1

print('total numbers of data in last 2 months:',length)

total numbers of data in last 2 months: 42


In [10]:
new_df=new_df1[0:length]
new_df.head()

,day,month,year,weekday,weekday_name,Close
0,4,1,2019,4,Friday,1098.65
1,3,1,2019,3,Thursday,1092.75
2,2,1,2019,2,Wednesday,1106.40
3,1,1,2019,1,Tuesday,1121.00
4,31,12,2018,0,Monday,1121.25


In [11]:
x=new_df[['month','day','weekday']].values
y=new_df['Close'].values

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=0)
reg=LinearRegression()
reg.fit(x_train,y_train)
y_pred=reg.predict(x_test)

In [13]:
from sklearn.linear_model import LinearRegression
X_train=new_df[['month','day','weekday']].values
Y_train=new_df['Close'].values
reg=LinearRegression()
reg.fit(X_train,Y_train)
new_df['Price_pred']=reg.predict(X_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
cur_price=[]
for i in range(len(new_df)):
    if new_df['Price_pred'][i]>new_df['Close'][i]:
        cur_price.append('Higher')
    else:
        cur_price.append('Lower')
new_df['Cur_price']=cur_price
new_df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,day,month,year,weekday,weekday_name,Close,Price_pred,Cur_price
0,4,1,2019,4,Friday,1098.65,1106.402878,Higher
1,3,1,2019,3,Thursday,1092.75,1104.847664,Higher
2,2,1,2019,2,Wednesday,1106.40,1103.292450,Lower
3,1,1,2019,1,Tuesday,1121.00,1101.737236,Lower
4,31,12,2018,0,Monday,1121.25,1125.195454,Higher
5,28,12,2018,4,Friday,1125.55,1128.056919,Higher
6,27,12,2018,3,Thursday,1120.20,1126.501705,Higher
7,26,12,2018,2,Wednesday,1098.35,1124.946491,Higher
8,24,12,2018,0,Monday,1089.15,1121.836063,Higher
9,21,12,2018,4,Friday,1100.20,1124.697528,Higher


In [15]:
x=new_df[['month','day','weekday']].values
y=new_df['Cur_price'].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [16]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
estimator = [] 
estimator.append(('SVC', SVC(gamma ='auto', probability = True))) 
estimator.append(('DTC', DecisionTreeClassifier()))
model = VotingClassifier(estimators = estimator, voting ='soft') 
model.fit(x_train, y_train) 
y_pred = model.predict(x_test)

In [17]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(reg, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))

In [18]:
filename = 'finalized_model.pkl1'
pickle.dump(model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))

In [1]:
pip install -U scikit-learn


     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [20]:
import sklearn
sklearn.__version__

'0.24.2'